In [17]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Función para cargar y preprocesar imágenes
def load_and_preprocess_images(data, img_size=(96, 96), path_col='path', label_col='label', filter_labels=None):
    images = []
    labels = []
    for index, row in data.iterrows():
        if filter_labels is None or row[label_col] in filter_labels:
            img_path = row[path_col].replace('..', 'data').replace('\\', '/')
            img = load_img(img_path, target_size=img_size, color_mode='rgb')  # Cambiado a 'rgb'
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(row[label_col])
    images = np.array(images, dtype='float32') / 255.0
    return images, labels

In [5]:
def load_and_preprocess_images2(data, img_size=(96, 96)):
    images = []
    for index, row in data.iterrows():
        img_path = f"data/data/images/test/{row['id_img']}.jpg"
        img = load_img(img_path, target_size=img_size, color_mode='rgb')  # Cambiado a 'rgb'
        img_array = img_to_array(img)
        images.append(img_array)
    images = np.array(images, dtype='float32') / 255.0
    return images

In [6]:
# Cargar los datos
train_data = pd.read_csv('data/train_set.csv')
test_data = pd.read_csv('test_set.csv')
sample_submission = pd.read_csv('sample_submision.csv')

In [7]:
test_images = load_and_preprocess_images2(test_data)

In [8]:
# Ajustar los nombres de las columnas según sea necesario
path_col = 'path'  # Cambia esto si el nombre de la columna es diferente
label_col = 'label'

In [9]:
# Mostrar las columnas de los DataFrames
print("Columnas de train_set.csv:", train_data.columns)
print("Columnas de test_set.csv:", test_data.columns)

Columnas de train_set.csv: Index(['id_img', 'path', 'label'], dtype='object')
Columnas de test_set.csv: Index(['id_img'], dtype='object')


In [10]:
# Cargar y procesar imágenes de entrenamiento
train_images, train_labels = load_and_preprocess_images(train_data, path_col=path_col, label_col=label_col, filter_labels=['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'])

In [11]:
# Codificar etiquetas para 7 clases
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
train_labels_categorical = to_categorical(train_labels_encoded, num_classes=7)

In [12]:
# Usar ResNet50 para extraer características
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

In [14]:
# Extraer características de las imágenes
features = model.predict(train_images)

901/901 ━━━━━━━━━━━━━━━━━━━━ 497s 548ms/step


In [15]:
# Dividir los datos en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(features, train_labels_encoded, test_size=0.2, random_state=42)

In [20]:
# Definir los modelos y sus hiperparámetros
models = [
    # ('RandomForest', RandomForestClassifier(), {
    #     'model__n_estimators': [50, 100, 200],
    #     'model__max_depth': [None, 10, 20, 30],
    #     'model__min_samples_split': [2, 5, 10],
    #     'model__min_samples_leaf': [1, 2, 4]
    # }),
    ('GradientBoosting', GradientBoostingClassifier(), {
        'model__n_estimators': [50, 100, 200],
        'model__max_depth': [3, 5, 10],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 4]
    }),
    ('SVC', SVC(), {
        'model__C': [0.1, 1, 10, 100],
        'model__gamma': [1, 0.1, 0.01, 0.001],
        'model__kernel': ['rbf', 'linear']
    }),
    ('KNN', KNeighborsClassifier(), {
        'model__n_neighbors': [3, 5, 11, 19],
        'model__weights': ['uniform', 'distance'],
        'model__metric': ['euclidean', 'manhattan']
    })
]

In [21]:
# Pipeline para escalar datos y probar modelos
for name, model, params in models:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    grid_search = GridSearchCV(pipeline, params, cv=3, n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    print(f"Mejores Parámetros para {name}: {best_params}")

    best_model = grid_search.best_estimator_
    val_predictions = best_model.predict(X_val)
    val_accuracy = accuracy_score(y_val, val_predictions)
    print(f"Validation accuracy for {name}: {val_accuracy:.4f}")

    # Si este modelo es el mejor hasta ahora, guárdalo
    if 'best_accuracy' not in locals() or val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model_name = name
        best_model = best_model
        best_model_params = best_params

# Mostrar el mejor modelo y sus parámetros
print(f"Mejor modelo: {best_model_name} con precisión de validación: {best_accuracy}")
print(f"Mejores parámetros: {best_model_params}")

Fitting 3 folds for each of 81 candidates, totalling 243 fits


KeyboardInterrupt: 

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Mejores Parámetros para RandomForest: {'model__max_depth': 30, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Validation accuracy for RandomForest: 0.4062

In [ ]:
# Extraer características de las imágenes de prueba
test_features = model.predict(test_images)

Mejores Parámetros: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [ ]:
# Realizar predicciones sobre el conjunto de prueba con el mejor modelo
test_predictions = best_model.predict(test_features)
test_predictions_decoded = label_encoder.inverse_transform(test_predictions)

RandomForestClassifier(min_samples_split=3, n_estimators=3000)

In [ ]:
# Generar el archivo de submission con el nombre del modelo
submission_name = f"submission_{best_model_name}.csv"
submission = sample_submission.copy()

In [ ]:
# Asegurar que el submission mantenga el formato correcto
submission['label'] = test_predictions_decoded

submission.to_csv(f'/mnt/data/{submission_name}', index=False)

print(f"Submission file saved as {submission_name}")

Validation accuracy: 0.4241


In [ ]:
submission